In [1]:
pip install fastestimator ~/repos/fledge

Processing /home/itrushkin/repos/fledge


  Created wheel for fledge: filename=fledge-0.1-py3-none-any.whl size=169166 sha256=a03c819a3726a213a4a1a886bd30d3e2318d3042b7a97cc687dffc8d00ca27e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-axqdl4v5/wheels/3a/30/9d/a88101effd240e4a8721301bebf95a245eaf4904b55b7ae530
Successfully built fledge
  Attempting uninstall: fledge
    Found existing installation: fledge 0.1
    Uninstalling fledge-0.1:
      Successfully uninstalled fledge-0.1
You should consider upgrading via the '/home/itrushkin/.virtualenvs/fledge/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fastestimator as fe
from fastestimator.architecture.pytorch import LeNet
from fastestimator.backend import to_tensor, argmax
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.meta import Sometimes
from fastestimator.op.numpyop.multivariate import HorizontalFlip
from fastestimator.op.numpyop.univariate import CoarseDropout, Normalize, Onehot
from fastestimator.op.tensorop import Average
from fastestimator.op.tensorop.gradient import Watch, FGSM
from fastestimator.op.tensorop.loss import CrossEntropy
from fastestimator.op.tensorop.model import ModelOp, UpdateOp
from fastestimator.trace.io import BestModelSaver
from fastestimator.trace.metric import Accuracy
from fastestimator.util import ImgData, to_number
from fledge.native.fastestimator import FederatedFastEstimator
from fastestimator.dataset.data import cifar10
from fastestimator.op.numpyop.univariate import Normalize, ChannelTranspose

#data_path='1'
#collaborator_count=2
batch_size=128

train_data, eval_data = cifar10.load_data()
test_data = eval_data.split(0.5)
#train_data, eval_data, test_data = split_data(train_data, eval_data, test_data, int(data_path), collaborator_count)

pipeline = fe.Pipeline(train_data=train_data,
                     eval_data=eval_data,
                     test_data=test_data,
                     batch_size=batch_size,
                     ops=[Normalize(inputs="x", outputs="x",
                             mean=(0.4914, 0.4822, 0.4465), 
                             std=(0.2471, 0.2435, 0.2616)),
                          ChannelTranspose(inputs="x", outputs="x")])
model = fe.build(model_fn=lambda: LeNet(input_shape=(3, 32, 32)), \
                         optimizer_fn="adam", model_name="adv_model")

network = fe.Network(ops=[
                Watch(inputs="x"),
                ModelOp(model=model, inputs="x", outputs="y_pred"),
                CrossEntropy(inputs=("y_pred", "y"), outputs="base_ce"),
                FGSM(data="x", loss="base_ce", outputs="x_adverse", epsilon=0.04),
                ModelOp(model=model, inputs="x_adverse", outputs="y_pred_adv"),
                CrossEntropy(inputs=("y_pred_adv", "y"), outputs="adv_ce"),
                Average(inputs=("base_ce", "adv_ce"), outputs="avg_ce"),
                UpdateOp(model=model, loss_name="avg_ce")
            ])

estimator = fe.Estimator(pipeline=pipeline,
                         network=network,
                         epochs=2,
                         traces=[Accuracy(true_key="y", pred_key="y_pred", output_name="clean_accuracy"),
                                Accuracy(true_key="y", pred_key="y_pred_adv", output_name="adversarial_accuracy")],
                         max_train_steps_per_epoch=None,
                         max_eval_steps_per_epoch=None,
                         monitor_names=["base_ce", "adv_ce"],
                         log_steps=1000)
fledge_estimator = FederatedFastEstimator(estimator, rounds=10)
model=fledge_estimator.fit()

Instructions for updating:
non-resource variables are not supported in the long term
Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── code
│   ├── keras_cnn.py
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── tfmnist_inmemory.py
├── agg_to_col_two_signed_cert.zip
├── plan
│   ├── cols.yaml
│   ├── data.yaml
│   ├── defaults
│   │   ├── tasks_keras.yaml
│   │   ├── tasks_tensorflow.yaml
│   │   ├── tasks_fast_estimator.yaml
│   │   ├── network.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── tasks_torch.yaml
│   │   ├── aggregator.yaml
│   │   ├── task_runner.yaml
│   │   └── data_loader.yaml
│   └── plan.yaml
├── requirements.txt
├── logs
├── agg_to_col_one_signed_cert.zip
├── cert
│   ├── config
│   │   ├── client.conf
│   │   ├── signing-ca.conf
│   │   ├── server.conf
│   │   └── root-ca.conf
│   ├── client
│   │   ├── col_one.crt
│   │   ├── col_two.csr
│   │   ├

/home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:172: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)


FastEstimator-Test: step: None; epoch: 2; clean_accuracy: 0.0984; adversarial_accuracy: 0.0984; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; adv_ce: 2.318904; base_ce: 2.3025656; avg_ce: 2.3107347; 
FastEstimator-Train: step: 98; epoch: 1; epoch_time: 2.86 sec; 
FastEstimator-Eval: step: 98; epoch: 1; adv_ce: 1.9409097; base_ce: 1.8045498; avg_ce: 1.8727299; clean_accuracy: 0.316; adversarial_accuracy: 0.2664; 
FastEstimator-Train: step: 196; epoch: 2; epoch_time: 2.88 sec; 
FastEstimator-Eval: step: 196; epoch: 2; adv_c

FastEstimator-Test: step: 196; epoch: 2; clean_accuracy: 0.4152; adversarial_accuracy: 0.3112; 


...

    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; adv_ce: 1.0872691; base_ce: 0.59845287; avg_ce: 0.84286094; 
FastEstimator-Train: step: 98; epoch: 1; epoch_time: 2.86 sec; 
FastEstimator-Eval: step: 98; epoch: 1; adv_ce: 1.9299662; base_ce: 1.1712519; avg_ce: 1.5506089; clean_accuracy: 0.5984; adversarial_accuracy: 0.392; 
FastEstimator-Train: step: 196; epoch: 2; epoch_time: 2.88 sec; 
FastEstimator-Eval: step: 196; epoch: 2; ad

FastEstimator-Test: step: 196; epoch: 2; clean_accuracy: 0.5968; adversarial_accuracy: 0.3728; 


FastEstimator-Test: step: 196; epoch: 2; clean_accuracy: 0.6; adversarial_accuracy: 0.388; 


    ______           __  ______     __  _                 __            
   / ____/___ ______/ /_/ ____/____/ /_(_)___ ___  ____ _/ /_____  _____
  / /_  / __ `/ ___/ __/ __/ / ___/ __/ / __ `__ \/ __ `/ __/ __ \/ ___/
 / __/ / /_/ (__  ) /_/ /___(__  ) /_/ / / / / / / /_/ / /_/ /_/ / /    
/_/    \__,_/____/\__/_____/____/\__/_/_/ /_/ /_/\__,_/\__/\____/_/     
                                                                        

FastEstimator-Warn: No ModelSaver Trace detected. Models will not be saved.
FastEstimator-Start: step: 1; num_device: 3; logging_interval: 1000; 
FastEstimator-Train: step: 1; adv_ce: 0.91773486; base_ce: 0.47613692; avg_ce: 0.6969359; 
FastEstimator-Train: step: 98; epoch: 1; epoch_time: 2.89 sec; 
FastEstimator-Eval: step: 98; epoch: 1; adv_ce: 1.862541; base_ce: 1.1263366; avg_ce: 1.4944388; clean_accuracy: 0.6112; adversarial_accuracy: 0.3832; 
FastEstimator-Train: step: 196; epoch: 2; epoch_time: 2.89 sec; 
FastEstimator-Eval: step: 196; epoch: 2; ad

FastEstimator-Test: step: 196; epoch: 2; clean_accuracy: 0.608; adversarial_accuracy: 0.364; 


In [3]:
model.state_dict()

OrderedDict([('module.conv1.weight',
              tensor([[[[ 1.0545e-01, -6.9001e-02, -5.8485e-02],
                        [ 1.7878e-01, -2.0313e-01, -9.0882e-02],
                        [-1.5487e-02, -2.2725e-01, -5.7066e-03]],
              
                       [[ 2.3951e-01, -9.4993e-02,  1.5313e-01],
                        [ 3.0232e-01,  4.9389e-02,  9.7059e-02],
                        [ 2.1739e-01,  3.9407e-02,  1.3037e-01]],
              
                       [[ 4.0230e-02, -1.0637e-01, -1.6270e-02],
                        [-3.6199e-02, -2.3833e-01, -1.3434e-01],
                        [-1.3303e-01, -1.7917e-01, -2.7975e-02]]],
              
              
                      [[[-2.7232e-01,  1.4038e-01,  1.5373e-01],
                        [-1.6211e-01,  1.6217e-01,  1.6370e-01],
                        [-1.1771e-01,  3.1811e-02,  1.8104e-02]],
              
                       [[-1.6862e-01,  1.2997e-01,  7.1054e-02],
                        [-2.9728e-01, 